# TextAttack End-to-End

This tutorial provides a broad end-to-end overview of training, evaluating, and attacking a model using [TextAttack](https://textattack.readthedocs.io/en/master/).

## Training
Text attack comes with its own fine tuned models on several datasets. You can list them with the command below.

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('universal_tagset')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jgaspar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jgaspar/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/jgaspar/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [2]:
!textattack list models

2025-10-05 21:34:38.282863: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-05 21:34:38.358062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759692878.387287  151906 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759692878.394290  151906 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759692878.453250  151906 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

You can use these models as it is by referring to their name. However for the purpose of this practical we are going to train our model from scratch.

TextAttack integrates directly with [transformers](https://github.com/huggingface/transformers/) and [datasets](https://github.com/huggingface/datasets) to train any of the `transformers` pre-trained models on datasets from `datasets`.

Let's use the Rotten Tomatoes Movie Review dataset: it's relatively short, and showcases the key features of `textattack train`. Let's take a look at the dataset using `textattack peek-dataset`:

In [3]:
!textattack peek-dataset --dataset-from-huggingface rotten_tomatoes

2025-10-05 21:34:44.339093: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-05 21:34:44.351238: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759692884.366055  152283 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759692884.370402  152283 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759692884.382630  152283 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

The dataset looks good! It's lowercased already, so we'll make sure our model is uncased. The longest input is 51 words, so we can cap our maximum sequence length (`--model-max-length`) at 64.

We'll train [`distilbert-base-uncased`](https://huggingface.co/transformers/model_doc/distilbert.html), since it's a relatively small model, and a good example of how we integrate with `transformers`.

So we have our command:

```bash
textattack train                      \ # Train a model with TextAttack
    --model distilbert-base-uncased   \ # Using distilbert, uncased version, from `transformers`
    --dataset rotten_tomatoes         \ # On the Rotten Tomatoes dataset
    --model-num-labels 2              \ # That has 2 labels
    --model-max-length 64             \ # With a maximum sequence length of 64
    --per-device-train-batch-size 128 \ # And batch size of 128
    --num-epochs 3                    \ # For 3 epochs
```

Now let's run it (please remember to use GPU if you have access):

In [4]:
!textattack train --model-name-or-path distilbert-base-uncased --dataset rotten_tomatoes --model-num-labels 2 --model-max-length 64 --per-device-train-batch-size 128 --num-epochs 3

2025-10-05 21:03:53.574192: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-05 21:03:53.585558: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759691033.598872  133694 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759691033.603222  133694 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759691033.614759  133694 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## Evaluation

We successfully fine-tuned `distilbert-base-cased` for 3 epochs. Now let's evaluate it using `textattack eval`. This is as simple as providing the path to the pretrained model (that you just obtain from running the above command!) to `--model`, along with the number of evaluation samples. `textattack eval` will automatically load the evaluation data from training:

In [4]:
!textattack eval --num-examples 1000 --model ./outputs/2025-10-05-21-03-56-141754/best_model/ --dataset-from-huggingface rotten_tomatoes --dataset-split test

2025-10-05 21:35:10.046523: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-05 21:35:10.058401: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759692910.072823  152780 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759692910.077463  152780 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759692910.089943  152780 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Awesome -- we were able to train a model up to 84.4% accuracy on the test dataset – with only a single command!

## Attack

Finally, let's attack our pre-trained model. We can do this the same way as before (by providing the path to the pretrained model to `--model`). For our attack, let's use the "TextFooler" attack recipe, from the paper ["Is BERT Really Robust? A Strong Baseline for Natural Language Attack on Text Classification and Entailment" (Jin et al, 2019)](https://arxiv.org/abs/1907.11932). We can do this by passing `--recipe textfooler` to `textattack attack`.

> *Warning*: We're printing out 100 examples and, if the attack succeeds, their perturbations. The output of this command is going to be quite long!


In [ ]:
!textattack attack --recipe textfooler --num-examples 100 --model ./outputs/2025-10-05-21-03-56-141754/best_model/ --dataset-from-huggingface rotten_tomatoes --dataset-split test 

2025-10-05 21:35:43.613430: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-05 21:35:43.625272: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759692943.641443  153321 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759692943.646131  153321 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759692943.658318  153321 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Looks like our model was 84% successful (makes sense - same evaluation set as `textattack eval`!), meaning that TextAttack attacked the model with 84 examples (since the attack won't run if an example is originally mispredicted). The attack success rate was 98.8%, meaning that TextFooler failed to find an adversarial example only 1.2% (1 out of 84) of the time.



#**TO DO: Robust models**

Now that we have trained our model and saw that it was vulnerable to adversarial attacks, your next task is to improve its robustness. We can do so by training a model with adversarial data instead of the normal ones.

1. To do so we need to update the training command from above to instruct textattack to use adversarial data generated with textfooler or any other attack during training.

To complete the task you can take the help of the documentation of [TextAttack library ](https://textattack.readthedocs.io/en/latest/0_get_started/basic-Intro.html) and command line help option to adversarially train a model on the same dataset.

***Hint***: Take a look at the [Trainer class in API](https://textattack.readthedocs.io/en/master/api/trainer.html) user guide and the [Making Vanilla Adversarial Training of NLP Models Feasible!](https://textattack.readthedocs.io/en/master/1start/A2TforVanillaAT.html)

2. If the solution you found is expected to take more than 15 minutes to train, look again at the docummentation and adapt your parameters such that it will take between 5-10 minutes due to time restrictions for this class.

3. Evaluate if the robustnes of the model has improved by attacking the newly trained model with TextFooler

In [6]:
!textattack train --help

2025-10-05 21:37:26.946080: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-05 21:37:26.958050: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759693046.972078  154449 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759693046.976652  154449 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759693046.988621  154449 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [5]:
  ### Modify the  following command to perform adversarial training

!textattack train \
    --model-name-or-path distilbert-base-uncased \
    --dataset rotten_tomatoes \
    --model-num-labels 2 \
    --model-max-length 64 \
    --per-device-train-batch-size 128 \
    --num-epochs 3 \
    --attack textfooler \
    --output-dir ./outputs/adversarial_model

2025-10-07 18:22:27.965032: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-07 18:22:27.975995: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759854147.990180   64652 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759854147.994864   64652 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759854148.006766   64652 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [6]:
!textattack eval --num-examples 1000 --model ./outputs/adversarial_model/best_model --dataset-from-huggingface rotten_tomatoes --dataset-split test

2025-10-07 19:18:59.138292: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-07 19:18:59.149399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759857539.163493  103867 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759857539.167846  103867 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759857539.183882  103867 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [7]:
!textattack attack --recipe textfooler --num-examples 100 --model ./outputs/adversarial_model/best_model/ --dataset-from-huggingface rotten_tomatoes --dataset-split test 

2025-10-07 19:28:32.535677: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-07 19:28:32.546797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759858112.561949  110709 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759858112.566760  110709 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759858112.578608  110709 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Attack results after robust training:
```
[Succeeded / Failed / Skipped / Total] 72 / 17 / 11 / 100: 100%|█| 100/100 [07:2

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 72     |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 11     |
| Original accuracy:            | 89.0%  |
| Accuracy under attack:        | 17.0%  |
| Attack success rate:          | 80.9%  |
| Average perturbed word %:     | 23.16% |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 140.34 |
+-------------------------------+--------+
```